<a href="https://colab.research.google.com/github/JVengava/CS6910-DeepLearningAssignments/blob/main/Assignment2/Part%20B/Assignment2_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#import keras

# keras pre-trained models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,  Model

import wandb

# data preprocessing
# empty the datasets forlder before downloading the dataset
%cd
%cd .keras/datasets/
!rm -r *

# download data from given url
dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
data_dir = tf.keras.utils.get_file('nature_12K', origin=dataset_url, extract=True)

# split data
%cd
%cd .keras/datasets/inaturalist_12K
%mv val test
!mkdir valid

# split train data into validation set and training set
data_folder = '/root/.keras/datasets/inaturalist_12K'
os.chdir(data_folder)

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 144 kB 37.4 MB/s 
     |████████████████████████████████| 181 kB 39.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1ddac2b838b49e420199b174254ef34219cee2a41ac54f714179d04ae6dcab03
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory
3816701952/3816687935 [==============================] - 30s 0us/step
/root
/root/.keras/datasets/inaturalist_12K


In [ ]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

# keras pre-trained models
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception


from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Conv2D, BatchNormalization, MaxPooling2D, Activation, GlobalAveragePooling2D 
from tensorflow.keras.models import Sequential,  Model

import wandb

# data preprocessing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib

#
import tensorflow as tf
#from modelClass import ObjectDetection


#wandb logging
from wandb.keras import WandbCallback


physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass




#data pre processing

data_augmentation = False

IMG_SIZE = (224,224)
BATCH_SIZE = 32


BASE_MODELS = {
                  "IRV2" : IRV2,
                  "IV3" : InceptionV3,
                  "RN50" : ResNet50,
                  "XCPTN" : Xception
              } 


sweep_config = {
  "name": "Bayesian Sweep Transfer Learning",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': [3],
        's': [2]
  },
  "parameters": {
                    

        "base_model": {
            "values": [ "XCPTN", "IV3", "RN50", "IRV2"]
        },
        "epochs": {
            "values": [ 5, 10, 3]
        }, 
        "dense_neurons": {
            "values": [ 128, 256]
        },
        "batch_size": {
            "values": [ 16, 32]
        },
        "optimizer": {
            "values": [ "adam","nadam","rmsprop"]
        } 
              
    }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910 _Assignment2', entity='cs21z032_cs22z005')


def load_pretrained_model():
        tf.keras.backend.clear_session()
        pretrained_model = BASE_MODELS["RN50"]
        new_input = Input(shape=(224, 224, 3), name="input")
        base = pretrained_model(weights='imagenet', input_tensor=new_input)
        #model = Model(inputs=base.input)
        model = Sequential([base, Flatten(), Dense(1000, activation='relu', kernel_initializer="he_uniform"), Dense(10, activation='softmax')])
        # freeze all base model's layers
        for layer in base.layers:
            layer.trainable = False

        return model

def load_pretrained_model_configurable(config):
        tf.keras.backend.clear_session()
        pretrained_model = BASE_MODELS[config["base_model"]]
        base = pretrained_model(weights='imagenet', include_top=False)
        X = base.output
        X = GlobalAveragePooling2D()(X)
        X = Dense(config["dense_neurons"], activation='sigmoid')(X)
        predictions = Dense(config["num_classes"], activation='softmax')(X)
        model = Model(inputs=base.input, outputs=predictions)

        # freeze all base model's layers
        for layer in base.layers:
            layer.trainable = False

        return model


def transfer_learn():
    config_defaults = dict(
                dense_neurons =256 ,
                activation = 'relu',
                num_classes = 10,
                optimizer = 'adam',
                epochs = 5,
                batch_size = 32, 
                img_size = (224,224),
                base_model = "IV3",
                data_augmentation = False
            ) 

    wandb.init( config = config_defaults)
    
    CONFIG = wandb.config


    wandb.run.name = "OBJDET_TransferLearn_" + CONFIG.base_model + "_dn_" + str(CONFIG.dense_neurons) + "_opt_" + CONFIG.optimizer + "_ep_" + str(CONFIG.epochs) + "_bs_"+str(CONFIG.batch_size) + "_act_" + CONFIG.activation

    data_augmentation = CONFIG.data_augmentation

    
    BATCH_SIZE = CONFIG.batch_size


    if data_augmentation == True:

    #Faster Alternative
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split = 0.1,
                shear_range=0.2,
                zoom_range=0.2,
                featurewise_center=False,
                samplewise_center=False,
                featurewise_std_normalization=False,
                samplewise_std_normalization=False,
                zca_whitening=False,
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=True,
                vertical_flip=False
                )
    else:
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split = 0.1)

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


    train_ds = train_datagen.flow_from_directory(
        data_folder+'/train',
        subset='training',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 123)
        
    validation_ds = train_datagen.flow_from_directory(
            data_folder+'/train',
            target_size=IMG_SIZE,
            subset = 'validation',
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)


            
    test_ds = test_datagen.flow_from_directory(
            data_folder+'/test',
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            shuffle = True,
            seed = 123)




    model = load_pretrained_model_configurable(CONFIG)
    #model = load_pretrained_model()
    model.summary()

    model.compile(
    optimizer=CONFIG.optimizer,  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=['accuracy'],
    )

    history = model.fit(
                    train_ds,
                    steps_per_epoch = train_ds.samples // CONFIG.batch_size,
                    validation_data = validation_ds, 
                    validation_steps = validation_ds.samples // CONFIG.batch_size,
                    epochs = CONFIG.epochs, 
                    callbacks=[WandbCallback()]
                    )
    model.evaluate(
                    test_ds,
                    batch_size = 32,
                    callbacks=[WandbCallback()]
                  )
                    
    model.save('./TransferLearning_TrainedModel/'+wandb.run.name)                
    wandb.finish()
    
    #wandb.agent(sweep_id,transfer_learn, count = 15)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. {'min_iter': [3], 's': [2], 'type': 'hyperband'} is not valid under any of the given schemas


Create sweep with ID: 174p8ce6
Sweep URL: https://wandb.ai/cs21z032_cs22z005/CS6910%20_Assignment2/sweeps/174p8ce6


In [ ]:
wandb.agent(sweep_id,transfer_learn, count = 100)

wandb: Agent Starting Run: bshg78e4 with config:
wandb: 	base_model: RN50
wandb: 	batch_size: 16
wandb: 	dense_neurons: 256
wandb: 	epochs: 5
wandb: 	optimizer: adam


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        

accuracy,▁▃▅▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▁▅▆█
val_loss,█▇▄▃▁
accuracy,0.19768
best_epoch,4
best_val_loss,2.16169
epoch,4
loss,2.18747
val_accuracy,0.23488


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: p3juh5xf with config:
wandb: 	base_model: XCPTN
wandb: 	batch_size: 32
wandb: 	dense_neurons: 128
wandb: 	epochs: 5
wandb: 	optimizer: adam


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
83697664/83683744 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  


...

wandb: Agent Starting Run: lghzay9d with config:
wandb: 	base_model: IRV2
wandb: 	batch_size: 16
wandb: 	dense_neurons: 128
wandb: 	epochs: 3
wandb: 	optimizer: rmsprop


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         

accuracy,▁▆█
epoch,▁▅█
loss,█▂▁
val_accuracy,▃▁█
val_loss,▁█▁
accuracy,0.79141
best_epoch,0
best_val_loss,0.72836
epoch,2
loss,0.63891
val_accuracy,0.78226


wandb: Agent Starting Run: 8b8kjxh4 with config:
wandb: 	base_model: RN50
wandb: 	batch_size: 32
wandb: 	dense_neurons: 256
wandb: 	epochs: 10
wandb: 	optimizer: rmsprop


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        